In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000

Dataset URL: https://www.kaggle.com/datasets/kmader/skin-cancer-mnist-ham10000
License(s): CC-BY-NC-SA-4.0
100% 5.18G/5.20G [00:23<00:00, 244MB/s]
100% 5.20G/5.20G [00:23<00:00, 233MB/s]


In [6]:
! mkdir data
! unzip -q skin-cancer-mnist-ham10000.zip -d data
! unzip -q /content/drive/MyDrive/HAM10000_segmentations_lesion_tschandl.zip -d data

In [7]:
! mkdir models

In [8]:
pip install -U albumentations

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 9.6 MB/s eta 0:00:00


In [9]:
import albumentations as A
import time
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
from tqdm import tqdm
from PIL import Image
from itertools import product
import torch.nn as nn
import copy
from albumentations.pytorch import ToTensorV2
import torch
import torch.nn.functional as Fun
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from torchvision import transforms

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix



In [10]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

Using device: cpu


In [11]:
DATA_DIR = 'data/'

METADATA_FILE = os.path.join(DATA_DIR, 'HAM10000_metadata.csv')
IMAGE_DIRS = [os.path.join(DATA_DIR, 'HAM10000_images_part_1'),
              os.path.join(DATA_DIR, 'HAM10000_images_part_2')]
MASK_DIR = os.path.join(DATA_DIR, 'HAM10000_segmentations_lesion_tschandl')

metadata = pd.read_csv(METADATA_FILE)

In [21]:
le = LabelEncoder()
metadata['dx_encoded'] = le.fit_transform(metadata['dx'])
num_classes = len(le.classes_)
print(f"Number of classes: {num_classes}")
print(le.classes_)
print(metadata['image_id'])

Number of classes: 7
['akiec' 'bcc' 'bkl' 'df' 'mel' 'nv' 'vasc']
0        ISIC_0027419
1        ISIC_0025030
2        ISIC_0026769
3        ISIC_0025661
4        ISIC_0031633
             ...     
10010    ISIC_0033084
10011    ISIC_0033550
10012    ISIC_0033536
10013    ISIC_0032854
10014    ISIC_0032258
Name: image_id, Length: 10015, dtype: object


In [12]:
def stratified_extracted_dataset(dataset, target, portion=0.2):
    # Calculate the number of samples to take from each class
    n_samples = int(len(target) * portion)

    # Group indices by class labels
    y_idx = defaultdict(list)
    for i, label in enumerate(target):
        y_idx[label].append(i)

    # Prepare the sampled dataset
    extracted_indices = []

    for label, indices in y_idx.items():
        np.random.shuffle(indices)  # Shuffle the indices for this class
        n_per_class = min(len(indices), n_samples // len(y_idx))  # Samples to take for this class
        extracted_indices.extend(indices[:n_per_class])  # Select the samples

    # Return the new dataset containing only the sampled indices
    return dataset.loc[extracted_indices]

In [13]:
class SkinCancerDataset(Dataset):
    def __init__(self, dataframe, image_dirs, mask_dir, transform=None):
        """
        Args:
            dataframe (pd.DataFrame): DataFrame containing image information and labels.
            image_dirs (list): List of directories where images are stored.
            mask_dir (str): Directory where segmentation masks are stored.
            transform (callable, optional): Transformations to apply to the images.
            mask_transform (callable, optional): Transformations to apply to the masks.
        """
        self.dataframe = dataframe.reset_index(drop=True)
        self.image_dirs = image_dirs
        self.mask_dir = mask_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def apply_mask(self, image, mask):
        # focus on the lesion area of the image
        return image * np.expand_dims(mask, axis=0)

    def __getitem__(self, idx):
        # Extract image ID and label
        img_id = self.dataframe.loc[idx, 'image_id']
        label = self.dataframe.loc[idx, 'dx_encoded']

        # Find image path
        img_path = None
        for dir in self.image_dirs:
            potential_path = os.path.join(dir, f"{img_id}.jpg")
            if os.path.exists(potential_path):
                img_path = potential_path
                break
        if img_path is None:
            raise FileNotFoundError(f"Image {img_id} not found in specified directories.")

        # Find mask path
        mask_path = os.path.join(self.mask_dir, f"{img_id}_segmentation.png")
        if not os.path.exists(mask_path):
            raise FileNotFoundError(f"Mask for image {img_id} not found in {self.mask_dir}.")

        # Load image
        image = np.array(Image.open(img_path).convert('RGB'))

        # Load mask
        mask = np.array(Image.open(mask_path).convert('L'))  # Grayscale

        # Apply transformations and masking to the image
        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        masked_image = self.apply_mask(image, mask)

        # Convert label to torch.long
        label = torch.tensor(label, dtype=torch.long)

        return image, label

In [14]:
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=20, p=0.5),
    A.Resize(224, 224),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

val_test_transform = A.Compose([
    A.Resize(224, 224),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])

In [15]:
def train_epoch(model, dataloader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        correct += torch.sum(preds == labels.data)
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct.double() / total

    return epoch_loss, epoch_acc

def eval_epoch(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels.data)
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct.double() / total

    return epoch_loss, epoch_acc

In [16]:
def alexnet(dr):
  model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=True)
  model.classifier[0] = nn.Dropout(p=dr)
  model.classifier[3] = nn.Dropout(p=dr)
  model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
  model.eval()
  model = model.to(DEVICE)

  # Define optimizer and scheduler
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
  scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

  return model, criterion, optimizer, scheduler

In [17]:
def run_epoch(model, optimizer, criterion, scheduler, train_loader, val_loader):
  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0
  num_epochs = 20

  for epoch in range(num_epochs):
      print(f'\nEpoch {epoch+1}/{num_epochs}')
      print('-' * 10)

      # Training Phase
      train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, DEVICE)
      print(f'Train Loss: {train_loss:.4f} Acc: {train_acc:.4f}')

      # Validation Phase
      val_loss, val_acc = eval_epoch(model, val_loader, criterion, DEVICE)
      print(f'Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}')

      # Check if this is the best model so far
      if val_acc > best_acc:
          best_acc = val_acc
          best_model_wts = copy.deepcopy(model.state_dict())
          torch.save(model.state_dict(), 'best_alexnet.pth')
          print(f'Best model updated with Val Acc: {best_acc:.4f}')

      # Step the scheduler
      scheduler.step()

  print('\nTraining complete.')
  print(f'Best Validation Accuracy: {best_acc:.4f}')

  return best_acc

# Load the best model weights
# model.load_state_dict(best_model_wts)

In [18]:
class stratifiedKFold:
  def __init__(self, n_split, shuffle, random_state):
    self.n_split = n_split
    self.shuffle = shuffle
    self.random_state = random_state

  def split(self, X, y):
    if (self.shuffle and self.random_state) is not None:
      np.random.seed(self.random_state)

    idx = np.arange(len(y))
    if self.shuffle:
      np.random.shuffle(idx)

    y_idx = defaultdict(list)
    for i, label in zip(idx, y):
      y_idx[label].append(i)

    splits = [[] for _ in range(self.n_split)]

    for label, i in y_idx.items():
      np.random.shuffle(i)
      split_portions = [len(i) // self.n_split] * self.n_split
      for j in range(len(i) % self.n_split):
        split_portions[j] += 1

      first = 0
      for k in range(self.n_split):
        last = first + split_portions[k]
        splits[k].extend(i[first:last])
        first = last

    for i in range(self.n_split):
      test_idx = splits[i]
      train_idx = np.concatenate([splits[j] for j in range(self.n_split) if j != i])
      yield train_idx, test_idx

    def get_n_splits(self):
        return self.n_split

In [19]:
def cross_validate(X, y, hyperparameters):
    outer_fold = stratifiedKFold(n_split=5, shuffle=True, random_state=42)
    outer_results = []

    # outer fold
    for train_idx, test_idx in outer_fold.split(X, y):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        inner_fold = stratifiedKFold(n_split=3, shuffle=True, random_state=42)
        best_acc = 0.0
        best_params = None

        # set up hyperparameter combinations
        param_keys = [list(d.keys())[0] for d in hyperparameters]
        param_values = [list(d.values())[0] for d in hyperparameters]
        combinations = product(*param_values)
        hyper_combo = [{param_keys[i]: combo[i] for i in range(len(param_keys))} for combo in combinations]

        for param in hyper_combo:
            dropout_rate = param['dr']
            model, criterion, optimizer, scheduler = alexnet(dropout_rate)
            inner_acc = []

            # inner fold
            for inner_train_idx, inner_val_idx in inner_fold.split(X_train, y_train):
                X_inner_train, X_inner_val = X_train.iloc[inner_train_idx], X_train.iloc[inner_val_idx]

                # TODO: replace/remove based on your own implementation
                # Create Datasets for inner folds
                train_dataset = SkinCancerDataset(X_inner_train, IMAGE_DIRS, MASK_DIR, transform=train_transform)
                val_dataset = SkinCancerDataset(X_inner_val, IMAGE_DIRS, MASK_DIR, transform=val_test_transform)

                # Create DataLoaders for inner folds
                BATCH_SIZE = 128
                train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
                val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

                best_inner_acc = run_epoch(model, optimizer, criterion, scheduler, train_loader, val_loader)
                inner_acc.append(best_inner_acc)

            mean_inner_acc = np.mean(inner_acc)

            # Update best score and parameters
            if mean_inner_acc > best_acc:
                best_acc = mean_inner_acc
                best_params = param
                print(best_params)

        # Retrain the model with the best parameters on the entire outer training set
        dropout_rate = best_params['dr']
        model, criterion, optimizer, scheduler = alexnet(dropout_rate)

        # Create Datasets for outer folds
        train_dataset = SkinCancerDataset(X_train, IMAGE_DIRS, MASK_DIR, transform=train_transform)
        test_dataset = SkinCancerDataset(X_test, IMAGE_DIRS, MASK_DIR, transform=val_test_transform)

        # Create DataLoaders for outer folds
        BATCH_SIZE = 128
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
        test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

        best_test_acc = 0.0

        best_outer_acc = run_epoch(model, optimizer, criterion, scheduler, train_loader, test_loader)

        if best_outer_acc > best_test_acc:
            best_test_acc = best_outer_acc

        outer_results.append((best_params, best_test_acc))

    hyper_result = []
    for params, acc in outer_results:
        hyper_result.append({'dr': params['dr'], 'accuracy': acc.item()})

    hyper_df = pd.DataFrame(hyper_result)
    file_path = 'drive/MyDrive/hyper_result.csv'
    hyper_df.to_csv(file_path, index=False)

    return outer_results

In [22]:
extracted = stratified_extracted_dataset(metadata, metadata['dx_encoded'], portion = 0.2)

In [ ]:
hyperparameters = [
    {'dr': [0.5, 0.6, 0.7]}
]

hyper_tuning_result = cross_validate(extracted, extracted['dx_encoded'], hyperparameters)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)



Epoch 1/20
----------
Train Loss: 1.7829 Acc: 0.3188
Val Loss: 1.4694 Acc: 0.4346
Best model updated with Val Acc: 0.4346

Epoch 2/20
----------
Train Loss: 1.3105 Acc: 0.5056
Val Loss: 1.2846 Acc: 0.5233
Best model updated with Val Acc: 0.5233

Epoch 3/20
----------
Train Loss: 1.0392 Acc: 0.5984
Val Loss: 1.1117 Acc: 0.5809
Best model updated with Val Acc: 0.5809

Epoch 4/20
----------
Train Loss: 0.8676 Acc: 0.6801
Val Loss: 1.1103 Acc: 0.5565

Epoch 5/20
----------
Train Loss: 0.7279 Acc: 0.7394
Val Loss: 1.0310 Acc: 0.6053
Best model updated with Val Acc: 0.6053

Epoch 6/20
----------
Train Loss: 0.6207 Acc: 0.7673
Val Loss: 1.0070 Acc: 0.6297
Best model updated with Val Acc: 0.6297

Epoch 7/20
----------
Train Loss: 0.5130 Acc: 0.8110
Val Loss: 1.0407 Acc: 0.6364
Best model updated with Val Acc: 0.6364

Epoch 8/20
----------
Train Loss: 0.4589 Acc: 0.8311
Val Loss: 1.0169 Acc: 0.6364

Epoch 9/20
----------
Train Loss: 0.4129 Acc: 0.8591
Val Loss: 1.0366 Acc: 0.6386
Best model up

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0



Epoch 1/20
----------
Train Loss: 1.6571 Acc: 0.3602
Val Loss: 1.3715 Acc: 0.4656
Best model updated with Val Acc: 0.4656

Epoch 2/20
----------
Train Loss: 1.2138 Acc: 0.5716
Val Loss: 1.1943 Acc: 0.5765
Best model updated with Val Acc: 0.5765

Epoch 3/20
----------
Train Loss: 0.9661 Acc: 0.6454
Val Loss: 1.1164 Acc: 0.5765

Epoch 4/20
----------
Train Loss: 0.7986 Acc: 0.7002
Val Loss: 0.9908 Acc: 0.6319
Best model updated with Val Acc: 0.6319

Epoch 5/20
----------
Train Loss: 0.7261 Acc: 0.7383
Val Loss: 1.0480 Acc: 0.6120

Epoch 6/20
----------
Train Loss: 0.6622 Acc: 0.7528
Val Loss: 1.0198 Acc: 0.6408
Best model updated with Val Acc: 0.6408

Epoch 7/20
----------
Train Loss: 0.5794 Acc: 0.8031
Val Loss: 0.9754 Acc: 0.6763
Best model updated with Val Acc: 0.6763

Epoch 8/20
----------
Train Loss: 0.4944 Acc: 0.8143
Val Loss: 0.9422 Acc: 0.6851
Best model updated with Val Acc: 0.6851

Epoch 9/20
----------
Train Loss: 0.4411 Acc: 0.8423
Val Loss: 0.9287 Acc: 0.6918
Best model up

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


...


Epoch 1/20
----------
Train Loss: 1.6700 Acc: 0.3426
Val Loss: 1.3406 Acc: 0.4912
Best model updated with Val Acc: 0.4912

Epoch 2/20
----------
Train Loss: 1.3058 Acc: 0.5317
Val Loss: 1.2008 Acc: 0.5575
Best model updated with Val Acc: 0.5575

Epoch 3/20
----------
Train Loss: 1.1486 Acc: 0.5573
Val Loss: 1.1304 Acc: 0.5619
Best model updated with Val Acc: 0.5619

Epoch 4/20
----------
Train Loss: 0.9946 Acc: 0.6285
Val Loss: 1.0385 Acc: 0.5752
Best model updated with Val Acc: 0.5752

Epoch 5/20
----------
Train Loss: 0.8688 Acc: 0.6763
Val Loss: 0.9985 Acc: 0.5885
Best model updated with Val Acc: 0.5885

Epoch 6/20
----------
Train Loss: 0.8023 Acc: 0.6974
Val Loss: 1.0197 Acc: 0.6106
Best model updated with Val Acc: 0.6106

Epoch 7/20
----------
Train Loss: 0.7099 Acc: 0.7486
Val Loss: 1.0279 Acc: 0.6195
Best model updated with Val Acc: 0.6195

Epoch 8/20
----------
Train Loss: 0.7278 Acc: 0.7386
Val Loss: 0.9689 Acc: 0.6195

Epoch 9/20
----------
Train Loss: 0.6104 Acc: 0.7831
Va

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0



Epoch 1/20
----------
Train Loss: 1.8337 Acc: 0.2714
Val Loss: 1.4878 Acc: 0.4381
Best model updated with Val Acc: 0.4381

Epoch 2/20
----------
Train Loss: 1.4611 Acc: 0.4394
Val Loss: 1.2587 Acc: 0.5465
Best model updated with Val Acc: 0.5465

Epoch 3/20
----------
Train Loss: 1.2441 Acc: 0.5439
Val Loss: 1.1650 Acc: 0.5354

Epoch 4/20
----------
Train Loss: 1.1694 Acc: 0.5662
Val Loss: 1.1079 Acc: 0.5531
Best model updated with Val Acc: 0.5531

Epoch 5/20
----------
Train Loss: 1.0618 Acc: 0.5984
Val Loss: 1.0546 Acc: 0.6018
Best model updated with Val Acc: 0.6018

Epoch 6/20
----------
Train Loss: 0.9722 Acc: 0.6452
Val Loss: 1.0587 Acc: 0.5841

Epoch 7/20
----------
Train Loss: 0.8923 Acc: 0.6563
Val Loss: 1.0471 Acc: 0.5907

Epoch 8/20
----------
Train Loss: 0.8850 Acc: 0.6685
Val Loss: 0.9910 Acc: 0.6018

Epoch 9/20
----------
Train Loss: 0.8283 Acc: 0.7030
Val Loss: 0.9612 Acc: 0.6040
Best model updated with Val Acc: 0.6040

Epoch 10/20
----------
Train Loss: 0.7900 Acc: 0.707

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0



Epoch 1/20
----------
Train Loss: 1.6309 Acc: 0.3871
Val Loss: 1.2953 Acc: 0.5476
Best model updated with Val Acc: 0.5476

Epoch 2/20
----------
Train Loss: 1.1369 Acc: 0.5751
Val Loss: 1.0579 Acc: 0.6042
Best model updated with Val Acc: 0.6042

Epoch 3/20
----------
Train Loss: 0.9357 Acc: 0.6432
Val Loss: 0.9592 Acc: 0.6429
Best model updated with Val Acc: 0.6429

Epoch 4/20
----------
Train Loss: 0.8043 Acc: 0.7061
Val Loss: 0.8920 Acc: 0.6875
Best model updated with Val Acc: 0.6875

Epoch 5/20
----------
Train Loss: 0.6733 Acc: 0.7498
Val Loss: 0.8161 Acc: 0.6935
Best model updated with Val Acc: 0.6935

Epoch 6/20
----------
Train Loss: 0.5856 Acc: 0.7942
Val Loss: 0.8512 Acc: 0.7024
Best model updated with Val Acc: 0.7024

Epoch 7/20
----------
Train Loss: 0.5237 Acc: 0.8113
Val Loss: 0.9261 Acc: 0.6964

Epoch 8/20
----------
Train Loss: 0.4451 Acc: 0.8446
Val Loss: 0.8151 Acc: 0.7113
Best model updated with Val Acc: 0.7113

Epoch 9/20
----------
Train Loss: 0.4001 Acc: 0.8579
Va

In [ ]:
hyper_tuning_result

[({'dr': 0.6}, tensor(0.7135, dtype=torch.float64)),
 ({'dr': 0.5}, tensor(0.7478, dtype=torch.float64)),
 ({'dr': 0.5}, tensor(0.7292, dtype=torch.float64)),
 ({'dr': 0.5}, tensor(0.6815, dtype=torch.float64)),
 ({'dr': 0.6}, tensor(0.7351, dtype=torch.float64))]

In [23]:
def run_epoch_with_prediction(model, optimizer, criterion, scheduler, train_loader, test_loader, fold_idx):
  RESULT_SAVE_DIR = 'drive/MyDrive/Alexnet_Result'  # Directory to save results
  best_acc = 0.0
  num_epochs = 20

  epoch_train_acc = []
  epoch_train_loss = []
  epoch_val_acc = []
  epoch_val_loss = []
  epoch_idx = []

  for epoch in range(num_epochs):
      print(f'\nEpoch {epoch+1}/{num_epochs}')
      print('-' * 10)

      # Training Phase
      train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, DEVICE)
      print(f'Train Loss: {train_loss:.4f} Acc: {train_acc:.4f}')

      # Validation Phase
      val_loss, val_acc = eval_epoch(model, test_loader, criterion, DEVICE)
      print(f'Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}')

      epoch_train_acc.append(train_acc)
      epoch_train_loss.append(train_loss)
      epoch_val_acc.append(val_acc)
      epoch_val_loss.append(val_loss)
      epoch_idx.append(epoch)

      # Check if this is the best model so far
      if val_acc > best_acc:
          best_acc = val_acc

      # Step the scheduler
      scheduler.step()

  print('\nTraining complete.')
  print(f'Best Validation Accuracy: {best_acc:.4f}')

  epoch_df = pd.DataFrame({'Epoch': epoch_idx, 'Train_Acc': epoch_train_acc, 'Train_Loss': epoch_train_loss,
                           'Val_Acc': epoch_val_acc, 'Val_Loss': epoch_val_loss})
  epoch_file_name = f"drive/MyDrive/Alexnet_Result/fold{fold_idx+1}_epochs.csv"
  epoch_df.to_csv(epoch_file_name, index=False)
  print(f"Fold {fold_idx+1} epochs train/val metrics saved to {epoch_file_name}")

  model.eval()
  predictions = []
  actuals = []
  with torch.no_grad():
      for images, labels in test_loader:
          images = images.to(DEVICE)
          outputs = model(images)
          _, preds = torch.max(outputs, 1)
          predictions.extend(preds.cpu().numpy())
          actuals.extend(labels.cpu().numpy())

  result_df = pd.DataFrame({'Prediction': predictions, 'Actual': actuals})
  file_name = f"{RESULT_SAVE_DIR}/fold{fold_idx+1}.csv"
  result_df.to_csv(file_name, index=False)
  print(f"Fold {fold_idx+1} predictions saved to {file_name}")

  return model, best_acc

In [27]:
def cross_validate_20_fold(X, y, param):
    MODEL_SAVE_DIR = 'drive/MyDrive/Alexnet_Model'  # Directory to save models
    fold = stratifiedKFold(n_split=5, shuffle=True, random_state=42)
    fold_results = []

    best_acc = 0.0

    dropout_rate = param

    for fold_idx, (train_idx, test_idx) in enumerate(fold.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model, criterion, optimizer, scheduler = alexnet(dropout_rate)

        # Create Datasets for outer folds
        train_dataset = SkinCancerDataset(X_train, IMAGE_DIRS, MASK_DIR, transform=train_transform)
        test_dataset = SkinCancerDataset(X_test, IMAGE_DIRS, MASK_DIR, transform=val_test_transform)

        # Create DataLoaders for outer folds
        BATCH_SIZE = 128
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, pin_memory=True)
        test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0, pin_memory=True)

        model, val_acc = run_epoch_with_prediction(model, optimizer, criterion, scheduler, train_loader, test_loader, fold_idx)

        if val_acc > best_acc:
            best_acc = val_acc
            best_model_path = os.path.join(MODEL_SAVE_DIR, 'best_model.pth')
            torch.save(model.state_dict, best_model_path)
            print(f"New best model saved from Fold {fold_idx+1} with validation accuracy: {val_acc:.4f}")

        fold_results.append((fold_idx+1, best_acc))

    return fold_results

In [28]:
file_path = 'drive/MyDrive/hyper_result.csv'
hyper_tuning_result = pd.read_csv(file_path)

In [29]:
mean_dr = hyper_tuning_result.groupby('dr')['accuracy'].mean().reset_index()
best_dr = mean_dr.loc[mean_dr['accuracy'].idxmax(), 'dr']
best_dr

0.6

In [ ]:
fold_result = cross_validate_20_fold(metadata, metadata['dx_encoded'], best_dr)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0



Epoch 1/20
----------
Train Loss: 0.8365 Acc: 0.7002
Val Loss: 0.6500 Acc: 0.7616

Epoch 2/20
----------
Train Loss: 0.6300 Acc: 0.7695
Val Loss: 0.6004 Acc: 0.7661

Epoch 3/20
----------
Train Loss: 0.5395 Acc: 0.7949
Val Loss: 0.5368 Acc: 0.7995

Epoch 4/20
----------
Train Loss: 0.4854 Acc: 0.8180
Val Loss: 0.5149 Acc: 0.7950

Epoch 5/20
----------
Train Loss: 0.4347 Acc: 0.8403
Val Loss: 0.4894 Acc: 0.8190

Epoch 6/20
----------
Train Loss: 0.4012 Acc: 0.8512
Val Loss: 0.4691 Acc: 0.8219

Epoch 7/20
----------
Train Loss: 0.3541 Acc: 0.8665
Val Loss: 0.4650 Acc: 0.8274

Epoch 8/20
----------
Train Loss: 0.2863 Acc: 0.8976
Val Loss: 0.4366 Acc: 0.8339

Epoch 9/20
----------
Train Loss: 0.2603 Acc: 0.9012
Val Loss: 0.4288 Acc: 0.8384

Epoch 10/20
----------
Train Loss: 0.2477 Acc: 0.9089
Val Loss: 0.4269 Acc: 0.8394

Epoch 11/20
----------
Train Loss: 0.2415 Acc: 0.9126
Val Loss: 0.4317 Acc: 0.8429

Epoch 12/20
----------
Train Loss: 0.2356 Acc: 0.9112
Val Loss: 0.4314 Acc: 0.8444



KeyboardInterrupt: 